# Import

In [11]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),".."))

import math
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pytorch_kinematics as pk
from urdf_parser_py.urdf import URDF

# Initialization

In [2]:
urdf_path = os.path.join(os.getcwd(),"..","urdf","grace.urdf")
robot = URDF.from_xml_file(urdf_path)

# URDF

In [3]:
# URDF Object Properties

for joint in robot.joints:
    print(joint.name)

torso
realsense_mount
chest_camera
neck_pitch
neck_yaw
head_pitch
eyes_pitch
lefteye_yaw
righteye_yaw


In [4]:
# Content Manipulation

for joint in robot.joints:
    if joint.name == 'realsense_mount':
        print('Joint Object:',joint.origin)
        # Getting
        print("Single Attribute:",joint.origin.position)
        # Setting
        print("Attribute Type:",type(joint.origin.position))
        joint.origin.position = [joint.origin.position[0], joint.origin.position[1], joint.origin.position[2]]
        print("Output:",joint.origin.position)

Joint Object: rpy:
- 0.0
- 0.0
- 0.0
xyz:
- 0.10749
- 0.0
- 0.44989
Single Attribute: [0.10749, 0.0, 0.44989]
Attribute Type: <class 'list'>
Output: [0.10749, 0.0, 0.44989]


In [6]:
# To XML string with filtering

temp_str = robot.to_xml_string()
words = temp_str.split()
words[5] = '>'
urdf_str = ' '.join(words)
print(urdf_str)

<?xml version="1.0" ?> <robot name="robot" > <link name="world"/> <link name="torso"> <visual> <origin xyz="0.0 0.0 0.26" rpy="0 0 0"/> <geometry> <box size="0.26 0.3 0.6"/> </geometry> <material name="robot_model"> <color rgba="0.3 0.3 0.3 0.4"/> </material> </visual> </link> <link name="realsense_mount"> <visual> <origin xyz="0.0 0.0 0.00625" rpy="0.0 0.0 0.0"/> <geometry> <box size="0.01485 0.001 0.015"/> </geometry> <material name="robot_camera"> <color rgba="0.0 0.1 0.3 0.6"/> </material> </visual> </link> <link name="realsense"> <visual> <origin xyz="-0.01485 -0.0325 0.0" rpy="0.0 0.0 0.0"/> <geometry> <box size="0.025 0.09 0.025"/> </geometry> <material name="robot_camera"> <color rgba="0.0 0.1 0.3 0.6"/> </material> </visual> </link> <link name="neck_py"/> <link name="neck"> <visual> <origin xyz="0.0 0.0 0.07" rpy="0 0 0"/> <geometry> <cylinder radius="0.045" length="0.14"/> </geometry> <material name="robot_model"> <color rgba="0.3 0.3 0.3 0.4"/> </material> </visual> </link> 

# Pytorch Kinematics

In [7]:
chain = pk.build_chain_from_urdf(urdf_str).to
chain.print_tree()

world
└── torso
    ├── realsense_mount
    │   └── realsense
    └── neck_py
        └── neck
            └── head
                └── eyes
                    ├── lefteye
                    └── righteye



'world\n└── torso\n    ├── realsense_mount\n    │   └── realsense\n    └── neck_py\n        └── neck\n            └── head\n                └── eyes\n                    ├── lefteye\n                    └── righteye\n'

In [10]:
print(chain.get_joint_parameter_names())

['realsense_mount', 'neck_pitch', 'neck_yaw', 'head_pitch', 'eyes_pitch', 'lefteye_yaw', 'righteye_yaw']


In [16]:
# Specify Joint Angles (radians)
joint_cmd = {
    'realsense_mount': 0.0,
    'neck_pitch': 0.0,
    'neck_yaw': 0.0,
    'head_pitch': math.radians(10),
    'eyes_pitch': 0.0,
    'lefteye_yaw': 0.0,
    'righteye_yaw': 0.0,
}

In [15]:
# Forward Kinematics
ret = chain.forward_kinematics(joint_cmd)
ret

{'world': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0., 0., 0.]])),
 'torso': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0., 0., 0.]])),
 'realsense_mount': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0.1075, 0.0000, 0.4499]])),
 'realsense': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0.1223, 0.0325, 0.4774]])),
 'neck_py': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[-0.0177,  0.0000,  0.5343]])),
 'neck': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[-0.0177,  0.0000,  0.5343]])),
 'head': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[-0.0177,  0.0000,  0.6660]])),
 'eyes': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0.0672, 0.0000, 0.7179]])),
 'lefteye': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0.0672, 0.0290, 0.7179]])),
 'righteye': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[ 0.0672, -0.0290,  0.7179]]))}

In [17]:
# Forward Kinematics
ret = chain.forward_kinematics(joint_cmd)
ret

{'world': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0., 0., 0.]])),
 'torso': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0., 0., 0.]])),
 'realsense_mount': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0.1075, 0.0000, 0.4499]])),
 'realsense': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[0.1223, 0.0325, 0.4774]])),
 'neck_py': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[-0.0177,  0.0000,  0.5343]])),
 'neck': Transform3d(rot=tensor([[1., 0., 0., 0.]]), pos=tensor([[-0.0177,  0.0000,  0.5343]])),
 'head': Transform3d(rot=tensor([[ 0.9962,  0.0000, -0.0872,  0.0000]]), pos=tensor([[-0.0177,  0.0000,  0.6660]])),
 'eyes': Transform3d(rot=tensor([[ 0.9962,  0.0000, -0.0872,  0.0000]]), pos=tensor([[0.0570, 0.0000, 0.7318]])),
 'lefteye': Transform3d(rot=tensor([[ 0.9962,  0.0000, -0.0872,  0.0000]]), pos=tensor([[0.0570, 0.0290, 0.7318]])),
 'righteye': Transform3d(rot=tensor([[ 0.9962,  0.0000, -0.0872,  0.0000]]), pos=tens

In [18]:
ret['lefteye'].get_matrix()

tensor([[[ 0.9848,  0.0000, -0.1736,  0.0570],
         [ 0.0000,  1.0000,  0.0000,  0.0290],
         [ 0.1736,  0.0000,  0.9848,  0.7318],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]])

In [22]:
ret['righteye'].get_matrix()

tensor([[[ 0.9848,  0.0000, -0.1736,  0.0570],
         [ 0.0000,  1.0000,  0.0000, -0.0290],
         [ 0.1736,  0.0000,  0.9848,  0.7318],
         [ 0.0000,  0.0000,  0.0000,  1.0000]]])